## Cosmological recombination - solving Saha equation

We find the temperature and redshift at recombination, i.e. when electrons and protons in early universe first combined into neutral hydrogen. We use very simplified textbook approach and compare with more accurate numbers at the end.

In [ ]:
%matplotlib inline
from scipy import *
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brentq
from scipy.special import zeta
import scipy.constants as const

In [ ]:
const.find('Boltz')

In [ ]:
kB = const.value('Boltzmann constant in eV/K')
me = const.value('electron mass energy equivalent in MeV')*const.mega  # m_e in eV
mp = const.value('proton mass')  # m_p in kg
Q = const.value('Rydberg constant times hc in eV')  # 13.6 eV
sige = const.value('Thomson cross section')
c = const.value('speed of light in vacuum')
G = const.value('Newtonian constant of gravitation')

In [ ]:
H0pc = 68  # Hubble constant in km/s/Mpc
H0 = H0pc * const.kilo / (const.mega*const.parsec)
h = H0pc/100.

In [ ]:
ecrit0 = 3*c**2*H0**2/(8*pi*G)  # critical density in J m^-3
eta = 0.61e-9   # baryon-to-photon number ratio
TCMB = 2.7255   # CMB temperature today in K
#
Ob0 = 0.048
OL0 = 0.69
Om0 = 0.31

Saha equation, giving fractional ionization of Hydrogen as a function of temperature. See e.g. B. Ryden, _Introduction to Cosmology_ (2nd ed.) Eq. (8.34).

In [ ]:
def saha(kT, X=0.5):
    """Saha quation 
       
       kT -- temperature times Boltzmann constant
        X -- fractional ionization  
    """
    prefac = 2*zeta(3,1)/pi**2  * (2*pi)**(3/2)  # = 3.84
    return (1-X)/X**2 - prefac*eta*(abs(kT)/me)**1.5*exp(Q/(kT))

For numerical root finding we use `scipy`'s most robust 1D algorithm. We define recombination as the moment when half of the hydrogen is neutral.

In [ ]:
kTrec = brentq(saha, 0.1, 1., args=0.5); kTrec

In [ ]:
Trec = kTrec/kB; Trec  # recombination temperature in K

In [ ]:
zrec = Trec/TCMB - 1; zrec  # recombination redshift

In [ ]:
def z(X):
    """Redshift z for given fractional ionization X"""
    kT = brentq(saha, 0.1, 1, args=X)
    return kT/(kB*TCMB) - 1

In [ ]:
z(0.5)  # check

In [ ]:
Xs = np.linspace(0.999, 0.0019, 100)
zs = []
for X in Xs:
    zs.append(z(X))

In [ ]:
fig, ax = plt.subplots(figsize=[7,6])
ax.plot(zs, Xs, color='red', linestyle='-')
ax.set_xlabel('z', fontsize=14)
ax.set_ylabel('X', fontsize=14)
ax.plot((z(Xs[0]), z(0.5)), (0.5, 0.5), 'g--', lw=1)
ax.plot((z(0.5), z(0.5)), (0, 0.5), 'g--', lw=1)
ax.set_xlim(zs[-1], zs[0])
ax.set_ylim(0,1)
ax.invert_xaxis()

## Photon decoupling

In [ ]:
nb0 = Ob0*ecrit0/(mp * c**2)  # baryon number density

In [ ]:
def X(z):
    """Fractional ionization at redshift z"""
    kT = kB * TCMB * (1+z)
    return brentq(lambda X: saha(kT, X), 0.001, 1)

In [ ]:
X(1379)  # check, should be 0.5

In [ ]:
def decouple(z):
    """Equation Gamma(z) = H(z)"""
    return X(z)*(1+z)**3*nb0*sige*c - H0*sqrt(Om0*(1+z)**3)

In [ ]:
brentq(decouple, 1100, 1300)

So decoupling is a distinct event, coming after recombination.

## Comparison to Weinberg's numbers

In Table 2.1 of _Cosmology_ using the same simplified approach he gets (last column, for $\Omega_B h^2 = 0.03$) ionization 0.971 at 4200 K and 0.00401 at 3000 K. Formulas here give:

In [ ]:
eta=(0.03/0.75**2)*8.7/1.67/4.11e8; eta   # eta corresponding to his 3rd column

In [ ]:
(z(0.971)+1)*TCMB   # Weinberg: 4200 K

In [ ]:
(z(0.00401)+1)*TCMB # Weinberg: 3000 K

So this is in good agreement.

Using careful detailed treatment of recombination physics (a la Peebles), in Table 2.2 Weinberg gets ionization of X=0.122, for z=1100 and T=3000 K. This is in agreement with <a href='https://en.wikipedia.org/wiki/Recombination_(cosmology)#The_effective_three-level_atom'>Wikipedia</a> which has 90% of neutrality at z=1070.

In [ ]:
z(0.122)  # Weinberg: 1100

In [ ]:
(z(0.122)+1)*TCMB # Weinberg: 3000 K

So simplified treatment is accurate to some 15 %.